# MDL DB roll decay content
What is the data in the roll decay database and roll damping database?

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import data
import copy
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Base, Model, LoadingCondition, Run, RolldecayLinear, RolldecayDirect, RolldecayNorwegian
from mdldb.tables import Min, Mean, Max, Std, Ship
from mdldb import mdl_to_evaluation
from evaluation.run_dynamic import RunDynamic
from evaluation.run_manoeuvring import RunZigZag

from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
#from rolldecay.equations_lambdify import calculate_acceleration, calculate_velocity
from sklearn.pipeline import Pipeline

import signal_lab
from sqlalchemy.inspection import inspect
import seaborn as sns
import docs

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
roll_decay_tests = db.session.query(Run).filter((Run.id==Min.run_id) & 
                                                (Run.test_type=='roll decay'))

In [ ]:
len(roll_decay_tests.all())

In [ ]:
ships = db.session.query(Ship).filter((Run.id==Min.run_id) & 
                                                (Run.test_type=='roll decay'))

In [ ]:
df_ships = pd.DataFrame()

for ship in ships:
    instance = inspect(ship)
    
    s = pd.Series()
                
    for key,item in instance.attrs.items():
        if not isinstance(item.value, list):
            if pd.notnull(item.value):
                s[key]=item.value
    
    df_ships = df_ships.append(s, ignore_index=True)

In [ ]:
mask = df_ships[['lpp','beam','BKL','BKB']].notnull().all(axis=1)
df_ships = df_ships.loc[mask].copy()

In [ ]:
df_ships.hist(column = ['lpp','beam','BKL','BKB'],bins=20)

In [ ]:
roll_decay_test = roll_decay_tests[0]

In [ ]:
tables = ['min','mean','max','std']
statistics = {}
for table in tables:
    statistics[table] = pd.read_sql_table(table_name=table, con=engine)

In [ ]:
plots = [['x0','y0','z0'],
         ['u','v','delta'],
         ['roll','pitch','psi'],
         ['p','q','r']]
nrows = len(plots)
ncols = 0
for subplots in plots:
    if len(subplots) > ncols:
        ncols = len(subplots)

fig,axes = plt.subplots(nrows=nrows, ncols=ncols)
fig.set_size_inches(15,15)

for row, subplots in enumerate(plots):
    for col, plot in enumerate(subplots):
        
        ax = axes[row,col]
        min = statistics['min']
        max = statistics['max']
        std = statistics['std']
        
        bins = 30
        min[plot].hist(label='min', bins=bins, ax=ax)
        max[plot].hist(label='max', bins=bins, ax=ax, alpha=0.5)
        std[plot].hist(label='std', bins=bins, ax=ax, alpha=0.25)
        
        
        ax.set_title(plot)

    ax.legend()
plt.tight_layout()

### Comments
* All values above are in model scale. 
* Speed up to 2 m/s in model scale
* Rudder angle ($\delta$) is not used during test, but can have fixed initial angle to keep the model on a straight course.
* Max roll can be up to 25 degrees!
* The pitch is static (no standard deviation) so it is basically trim due to speed
* p, q and r are rotational velocities for roll, pitch and yaw.


In [ ]:
cols = ['roll','psi','u','r']
min[cols].describe()

In [ ]:
max[cols].describe()

In [ ]:
std[cols].describe()

In [ ]:
rolldecay_linear = pd.read_sql_table(table_name='rolldecay_linear', con=engine, index_col='run_id')
rolldecay_direct = pd.read_sql_table(table_name='rolldecay_direct', con=engine, index_col='run_id')
rolldecay_norwegian = pd.read_sql_table(table_name='rolldecay_norwegian', con=engine, index_col='run_id')
rolldecay_direct_improved = pd.read_sql_table(table_name='rolldecay_direct_improved', con=engine, index_col='run_id')


def remove_outliers(df, abs_percentile=0.01, min_score=0.97):
    mask1 = ((df.quantile(abs_percentile) < df) &
            (df < df.quantile(1-abs_percentile)) 
           )
    
    mask2 = df['score'] >= min_score
    return df[mask1].loc[mask2].copy()

rolldecay_linear = remove_outliers(rolldecay_linear)
rolldecay_direct = remove_outliers(rolldecay_direct)
rolldecay_norwegian = remove_outliers(rolldecay_norwegian)
rolldecay_direct_improved = remove_outliers(rolldecay_direct_improved)


In [ ]:
plots = [['zeta','d'],
         ['mean_damping','omega0'],
         ['score'],
        ]
nrows = len(plots)
ncols = 0
for subplots in plots:
    if len(subplots) > ncols:
        ncols = len(subplots)

fig,axes = plt.subplots(nrows=nrows, ncols=ncols)
fig.set_size_inches(15,15)

for row, subplots in enumerate(plots):
    for col, plot in enumerate(subplots):
        
        ax = axes[row,col]
        
        bins = 30
        rolldecay_linear[plot].hist(label='linear', bins=bins, ax=ax)
        rolldecay_direct[plot].hist(label='direct', bins=bins, ax=ax, alpha=0.25)
        rolldecay_norwegian[plot].hist(label='norwegian', bins=bins, ax=ax, alpha=0.25)
        rolldecay_direct_improved[plot].hist(label='dircet improved', bins=bins, ax=ax, alpha=0.25)
        
        
        ax.set_title(plot)

    ax.legend()
plt.tight_layout()

In [ ]:
plots= ['zeta','d','mean_damping','omega0','score']

labels = {
    'zeta':'$\zeta$',
    'd':'$d$',
    'omega0':'$\omega_0$',
}

plt.tight_layout()

for plot in plots:
    
    fig,ax = plt.subplots()
    fig.set_size_inches(1.6*np.array([5.7, 4.0]))
       
    bins = 50
    rolldecay_linear[plot].hist(label='linear', bins=bins, ax=ax)
    rolldecay_direct_improved[plot].hist(label='direct improved', bins=bins, ax=ax, alpha=0.5)    
    label = labels.get(plot,plot)
    ax.set_title('Histogram %s' % label)
    ax.set_xlabel(label)
    ax.set_ylabel('Number of tests')
    ax.legend();
    
    file_name = 'histogram_%s.png' % plot
    save_path = os.path.join(docs.path_presentation1,file_name)
    fig.savefig(save_path,transparent=True, dpi=100, bbox_inches='tight')


### Comments

So we now know that mean_damping is in the range 0-0.15...

In [ ]:
rolldecay_linear.describe()

In [ ]:
rolldecay_direct.describe()

In [ ]:
rolldecay_norwegian.describe()

In [ ]:
rolldecay_direct_improved.describe()

In [ ]:
for key in ['mean_damping','zeta','d']:
    
    fig,ax = plt.subplots()
    x = rolldecay_norwegian[key]
    y = rolldecay_direct.loc[rolldecay_norwegian.index][key]
    y2 = rolldecay_linear.loc[rolldecay_norwegian.index][key]    
    y3 = rolldecay_direct_improved.loc[rolldecay_norwegian.index][key]
    
    ax.plot(x,y,'.', label='direct')
    ax.plot(x,y2,'o', label='linear', alpha=0.5)
    ax.plot(x,y3,'o', label='direct improved', alpha=0.5)
    
    ax.set_xlabel('Norwegian: %s' % key)
    ax.legend();
    ax.plot(x,x,'r-')
    


### Comments
* Norwegian method and direct method give differnet results which is not very satisfying.
* Linear method gives higher mean damping, which is a bit strange.

In [ ]:
sql="""
SELECT * from
rolldecay_norwegian
INNER JOIN run
ON rolldecay_norwegian.run_id == run.id
    INNER JOIN loading_conditions
    ON (run.loading_condition_id == loading_conditions.id)
        INNER JOIN models
        ON run.model_number == models.model_number
            INNER JOIN ships
            ON models.ship_name == ships.name

"""
df_rolldecay = pd.read_sql(sql, con=engine, index_col='run_id',)
df_rolldecay = df_rolldecay.loc[:,~df_rolldecay.columns.duplicated()]
df_rolldecay = remove_outliers(df_rolldecay)
df_rolldecay.describe()

In [ ]:
df_rolldecay.columns

In [ ]:
df_rolldecay.describe()

In [ ]:
x_vars = ['lpp','beam','omega0','ship_speed','kg','gm']
sns.pairplot(df_rolldecay, x_vars=x_vars, y_vars=['mean_damping','omega0'],
             height=5, aspect=.8, kind="reg");

In [ ]:
df_rolldecay.columns

In [ ]:
df_rolldecay.isnull().sum().sort_values(ascending=False)

In [ ]:
important = ['KXX',                        
'BKL',                        
'BKX',                        
'RH',                         
'CW',                         
'CP',                         
'A0',                         
'LOA',                        
'AR',                         
'D',                          
'lpp',                        
'ship_speed',                 
'Volume',                     
'gm',                         
'TA',                         
'TF',                         
'beam',                       
'kg',                         
'lcg']                        
mask = df_rolldecay[important].notnull().all(axis=1)
df = df_rolldecay.loc[mask].copy()

In [ ]:
data = df[important].copy()

In [ ]:
df = df.dropna(subset=['omega0'])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

polynomial_features = PolynomialFeatures(degree=3)
linear_regression = LinearRegression()
model = make_pipeline(polynomial_features, linear_regression)

X=data
y=df['omega0']

model.fit(X=X, y=y)
model.score(X=X, y=y)

In [ ]:
fig,ax = plt.subplots()
ax.plot(y,model.predict(X),'.', alpha=0.5)

In [ ]:
fig,ax=plt.subplots()
(data['KXX']/data['beam']).hist(bins=50, ax=ax)
ax.set_title('Histogram of $K_{xx}$')
ax.set_xlabel('$K_{xx}$ [-]');
ax.set_ylabel('Occurences [1]')

In [ ]:
GM = df['gm']
beam = df['beam']
rxx = df['KXX']
kxx = rxx/beam
rho = 1000
m = rho*df['Volume']
Ixx = m*rxx**2
g = 9.81
omega0 = df['omega0']
Ixx_tot = GM*g*m/(omega0**2)
r_xx_tot = np.sqrt(Ixx_tot/m)
kxx_tot = r_xx_tot/beam

Ixx_added = Ixx_tot - Ixx
Ixx_added_ratio = Ixx_added/Ixx

rxx_added = r_xx_tot - rxx
rxx_added_ratio = rxx_added/rxx

fig,ax = plt.subplots()
Ixx_tot.hist(ax=ax, bins=50, label='$I_{xxtot}$')
Ixx.hist(ax=ax, bins=50, label='$I_{xx}$')
ax.legend()

fig,ax = plt.subplots()
kxx_tot.hist(ax=ax, bins=50, label='$k_{xxtot}$')
kxx.hist(ax=ax, bins=50, label='$k_{xx}$')
ax.legend()

fig,ax = plt.subplots()
Ixx_added_ratio.hist(ax=ax, bins=50)
ax.set_title('Added roll inertia ratio')

fig,ax = plt.subplots()
rxx_added_ratio.hist(ax=ax, bins=50)
ax.set_title('Added $r_{xx}$ ratio')


In [ ]:
data.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

polynomial_features = PolynomialFeatures(degree=3)
linear_regression = LinearRegression()
model = make_pipeline(polynomial_features, linear_regression)

X=data
y=df['mean_damping']

model.fit(X=X, y=y)
model.score(X=X, y=y)

In [ ]:
data.columns